In [1]:
# Load the model
from keras.models import load_model
import os
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint

Using TensorFlow backend.


In [2]:
model_file = os.path.join('..', 'model', 'grammy_prediction_model.h5')

grammy_prediction_model = load_model(model_file)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


- 'Year', 
- 'Acousticness', 
- 'Danceability', 
- 'Duration (ms)', 
- 'Energy',
- 'Instrumentalness', 
- 'Key', 
- 'Liveness', 
- 'Loudness', 
- 'Mode',
- 'Speechiness',
- 'Tempo',
- 'Time Signature',
- 'Valence',
- 'Explicit',
- 'Popularity'

In [3]:
# Spotify API Keys
spotify_cliend_id='bca78196e824433fbdf88ec18d84825f'
spotify_client_secret='d43763215bd8435eb9b3faaf048ca038'
os.environ['SPOTIPY_CLIENT_ID']=spotify_cliend_id
os.environ['SPOTIPY_CLIENT_SECRET']=spotify_client_secret

In [4]:
# Generate API Token
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
def get_features(title, artist, year=None):
    year_part=f"year:{year}" if year is not None else ''
    artist = str(artist)
    if ',' in artist:
        artists = artist.split(',')
        for artist in artists:
            features = get_features(title, artist)
            if features is not None:
                return features
            
    results = sp.search(q=f'track:{title} artist:{artist} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'{title} artist:{artist} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'{title} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'artist:{artist} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        return
    track_info = results['tracks']['items'][0]
    track_id = track_info['id']
    features = sp.audio_features([track_id])[0]
    
    if features is None:
        return
    
    #pprint(track_info)
    
    features_as_dict = {
        'Year': track_info['album']['release_date'][:4],
        'Acousticness': features['acousticness'],
        'Danceability': features['danceability'],
        'Duration (ms)': features['duration_ms'],
        'Energy': features['energy'],
        'Instrumentalness': features['instrumentalness'],
        'Key': features['key'],
        'Liveness': features['liveness'],
        'Loudness': features['loudness'],
        'Mode': features['mode'],
        'Speechiness': features['speechiness'],
        'Tempo': features['tempo'],
        'Time Signature': features['time_signature'],
        'Valence': features['valence'],
        'Explicit': track_info['explicit'],
        'Popularity': track_info['popularity']

    }
    return features_as_dict

In [6]:
test_data = [
    get_features('Adele', 'Someone like you'),
    get_features('Adele', 'Hello'),
    get_features('Stressed Out', 'twenty one pilots'),
    get_features('Dystopia', 'Megadeth'),
    get_features('Despacito', 'Luis Fonsi'),
    get_features("That's What I Like", 'Bruno Mars'),
    get_features("Volare", 'Domenico Modugno'),
    get_features("Beat it", 'Michael Jackson'),
    get_features("Catch A Falling Star", 'Perry Como'),
]
expected_results = [1, 0, 1, 1, 0, 1, 1, 1, 1]
test_data

[{'Year': '2011',
  'Acousticness': 0.892,
  'Danceability': 0.559,
  'Duration (ms)': 285040,
  'Energy': 0.33,
  'Instrumentalness': 0,
  'Key': 9,
  'Liveness': 0.0975,
  'Loudness': -8.251,
  'Mode': 1,
  'Speechiness': 0.0285,
  'Tempo': 135.109,
  'Time Signature': 4,
  'Valence': 0.285,
  'Explicit': False,
  'Popularity': 77},
 {'Year': '2017',
  'Acousticness': 0.862,
  'Danceability': 0.442,
  'Duration (ms)': 286000,
  'Energy': 0.379,
  'Instrumentalness': 0,
  'Key': 9,
  'Liveness': 0.103,
  'Loudness': -6.158,
  'Mode': 1,
  'Speechiness': 0.0299,
  'Tempo': 134.487,
  'Time Signature': 4,
  'Valence': 0.197,
  'Explicit': False,
  'Popularity': 1},
 {'Year': '2015',
  'Acousticness': 0.0462,
  'Danceability': 0.734,
  'Duration (ms)': 202333,
  'Energy': 0.637,
  'Instrumentalness': 2.29e-05,
  'Key': 4,
  'Liveness': 0.0602,
  'Loudness': -5.677,
  'Mode': 0,
  'Speechiness': 0.141,
  'Tempo': 169.977,
  'Time Signature': 4,
  'Valence': 0.648,
  'Explicit': False,
  '

In [7]:
df = pd.DataFrame(test_data)[['Year', 'Acousticness', 'Danceability', 'Duration (ms)', 'Energy',
       'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode',
       'Speechiness', 'Tempo', 'Time Signature', 'Valence', 'Explicit',
       'Popularity']]

df['Explicit'] = df.Explicit.astype(int)
df['Year'] = df.Year.astype(int)
excluded_features = ['Year', 'Key']
df.drop(labels=excluded_features, axis=1, inplace=True)
df


,Acousticness,Danceability,Duration (ms),Energy,Instrumentalness,Liveness,Loudness,Mode,Speechiness,Tempo,Time Signature,Valence,Explicit,Popularity
0,0.89200,0.559,285040,0.330,0.000000,0.0975,-8.251,1,0.0285,135.109,4,0.285,0,77
1,0.86200,0.442,286000,0.379,0.000000,0.1030,-6.158,1,0.0299,134.487,4,0.197,0,1
2,0.04620,0.734,202333,0.637,0.000023,0.0602,-5.677,0,0.1410,169.977,4,0.648,0,83
3,0.00006,0.274,299107,0.989,0.019300,0.0791,-2.895,0,0.4830,109.985,4,0.148,0,52
4,0.22800,0.653,228827,0.816,0.000000,0.0967,-4.353,1,0.1670,178.086,4,0.816,0,81
5,0.01350,0.845,204773,0.569,0.000000,0.0898,-5.127,1,0.0414,134.047,4,0.848,0,35
6,0.96300,0.297,274867,0.208,0.000022,0.1420,-15.611,0,0.0446,97.044,4,0.341,0,37
7,0.04910,0.779,258040,0.867,0.000008,0.1970,-3.704,0,0.0457,138.858,4,0.915,0,77
8,0.86400,0.628,148493,0.172,0.000000,0.1130,-17.816,1,0.0600,118.895,3,0.475,0,44


In [8]:
from sklearn.externals import joblib
scaler_file = os.path.join('..', 'model', 'grammy_prediction_scaler.sav')

X_scaler = joblib.load(scaler_file) 
df_scaled = X_scaler.transform(df)

/Users/kishore/Tools/anaconda3/envs/PythonStuff/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [9]:
df_scaled

array([[ 1.77864784,  0.02717856,  0.24903308, -1.10178373, -0.39927372,
        -0.56141361,  0.22425542,  0.64008495, -0.45510701,  0.51399635,
         0.23686502, -0.96347552, -0.26259675,  1.69697187],
       [ 1.68456984, -0.65731457,  0.25581351, -0.90020601, -0.39927372,
        -0.52994991,  0.65772955,  0.64008495, -0.44014378,  0.49287525,
         0.23686502, -1.29987476, -0.26259675, -1.90791461],
       [-0.87372452,  1.05099308, -0.33512219,  0.16116242, -0.3991798 ,
        -0.77479476,  0.75734783, -1.56229264,  0.74729531,  1.6980003 ,
         0.23686502,  0.42417135, -0.26259675,  1.98156817],
       [-1.01841648, -1.64017651,  0.34838757,  1.60923097, -0.32011994,
        -0.66667402,  1.3335184 , -1.56229264,  4.40259834, -0.33913303,
         0.23686502, -1.48718797, -0.26259675,  0.51115395],
       [-0.30361185,  0.57711322, -0.14799643,  0.89753819, -0.39927372,
        -0.56599015,  1.03155697,  0.64008495,  1.02518384,  1.97335559,
         0.23686502,  1.06

In [10]:
predictions = grammy_prediction_model.predict_classes(df_scaled)
predictions

array([1, 0, 0, 0, 1, 0, 1, 1, 1])

In [11]:
from keras.utils import to_categorical
import numpy as np

metric_names = grammy_prediction_model.metrics_names
metric_values = grammy_prediction_model.evaluate(df_scaled, to_categorical(np.asarray(expected_results)))
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

9/9 [==============================] - 0s 4ms/step
loss :  2.4225847721099854
acc :  0.5555555820465088
